In [1]:
from importlib import reload
#reload(Utilities)
#reload(clm)
# NOTE: To reload a class imported as, e.g., 
# from module import class
# One must call:
#   1. import module
#   2. reload module
#   3. from module import class

import sys, os
import re
from pathlib import Path
import json
import pickle

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype, is_timedelta64_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns, natsort_keygen
from packaging import version
import copy

import itertools

import pyodbc
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
import matplotlib.colors as mcolors
import matplotlib.cm as cm #e.g. for cmap=cm.jet
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
import CommonLearningMethods as clm
#-----
from MeterPremise import MeterPremise
#-----
from AMI_SQL import AMI_SQL
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from AMIEDE_DEV import AMIEDE_DEV
from MECPODf import MECPODf
from MECPOAn import MECPOAn
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
#sys.path.insert(0, os.path.join(os.path.realpath('..'), 'Utilities'))
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
from Utilities_df import DFConstructType
import Utilities_dt
import Plot_General
import Plot_Box_sns
import Plot_Hist
import Plot_Bar
import GrubbsTest
import DataFrameSubsetSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSlicer as DFSlicer

In [3]:
save_dir_base_raw = r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\regex_check\EndEvents'
save_dir_base_cur = r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\regex_check\EndEvents_curated'
date_ranges = [
    ['2023-04-01', '2023-04-07'], 
    ['2023-04-08', '2023-04-14'], 
    ['2023-04-15', '2023-04-21'], 
    ['2023-04-22', '2023-04-28'], 
    ['2023-04-29', '2023-05-05'], 
    ['2023-05-06', '2023-05-12'], 
    ['2023-05-13', '2023-05-19'], 
    ['2023-05-20', '2023-05-26'], 
    ['2023-05-27', '2023-05-31'], 
]
opcos = [
    'ap', 
    'oh', 
    'im', 
    'pso', 
    'swp', 
    'tx'
]

In [4]:
# for opco in opcos:
#     for date_range in date_ranges:
#         end_events_sql_function_kwargs=dict(
#             date_range=date_range, 
#             opco=opco
#         )
#         save_name = f"end_events_{opco}_{date_range[0].replace('-','')}_{date_range[1].replace('-','')}.pkl"
#         #-----
#         end_events = AMIEndEvents(
#             df_construct_type=DFConstructType.kRunSqlQuery, 
#             contstruct_df_args = None, 
#             build_sql_function=AMIEndEvents_SQL.build_sql_end_events, 
#             build_sql_function_kwargs=end_events_sql_function_kwargs, 
#             init_df_in_constructor=True, 
#             save_args=False
#         )
#         end_events.df.to_pickle(os.path.join(save_dir_base_raw, save_name))

In [5]:
# for opco in opcos:
#     for date_range in date_ranges:
#         save_name = f"end_events_{opco}_{date_range[0].replace('-','')}_{date_range[1].replace('-','')}.pkl"
#         ede_df = pd.read_pickle(os.path.join(save_dir_base_raw, save_name))
#         #-----
#         ede_df = AMIEndEvents.reduce_end_event_reasons_in_df(
#             df=ede_df, 
#             reason_col='reason', 
#             edetypeid_col='enddeviceeventtypeid',  
#             placement_col='curated_reason'
#         )
#         #-----
#         ede_df.to_pickle(os.path.join(save_dir_base_cur, save_name))

In [ ]:
ede_df = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\regex_check\EndEvents_curated\end_events_ap_20230401_20230407.pkl')

In [ ]:
freq='D'
group_cols=['serialnumber', 'aep_premise_nb']
group_freq=pd.Grouper(freq=freq, key='aep_event_dt')

In [ ]:
ede_df['aep_event_dt'] = pd.to_datetime(ede_df['aep_event_dt'])

In [ ]:
dev_ede_df = ede_df.iloc[:10000]

In [ ]:
dev_ede_df[['serialnumber', 'aep_premise_nb', 'curated_reason', 'aep_opco']].pivot_table(index=['serialnumber', 'aep_premise_nb'], columns='curated_reason', aggfunc='count')

In [ ]:
grp_by_full = group_cols+[group_freq]

In [ ]:
dev_ede_df.groupby(grp_by_full+['curated_reason']).sum()

In [ ]:
idk = dev_ede_df.groupby(grp_by_full+['curated_reason']).size().to_frame()
idk

In [ ]:
idk = idk.reset_index()
idk[idk['serialnumber']=='575731472']

In [ ]:
idk_i = idk[idk['serialnumber']=='575731472'].copy()
idk_i

In [ ]:
idk_i.pivot(index=['serialnumber', 'aep_premise_nb', 'aep_event_dt'], columns='curated_reason')

In [ ]:
hmm = idk.pivot(index=['serialnumber', 'aep_premise_nb', 'aep_event_dt'], columns='curated_reason').fillna(0)
hmm = hmm.droplevel(0, axis=1)
hmm

In [ ]:
# ede_df = ede_df[
#     (ede_df['aep_event_dt']>=pd.to_datetime('2023-04-01')) & 
#     (ede_df['aep_event_dt']<=pd.to_datetime('2023-04-02'))
# ]

In [ ]:
# ede_df = ede_df[ede_df['serialnumber'].isin(['585382540',
#  '585382026',
#  '578990289',
#  '698021317',
#  '590246217',
#  '584920788',
#  '578545196',
#  '578989278',
#  '590243062',
#  '578545604',
#  '594470889',
#  '578544961',
#  '594470869',
#  '594142342',
#  '585382633',
#  '590241689',
#  '587973848',
#  '590084483',
#  '584920790'])]

In [ ]:
start = time.time()
rcpo = AMIEndEvents.get_reason_counts_per_group(
    end_events_df=dev_ede_df, 
    group_cols=group_cols, 
    group_freq=group_freq, 
    reason_col='curated_reason', 
    return_form = dict(return_multiindex_outg_reason = False, 
                       return_normalized_separately  = False)   
)
print(time.time()-start)

In [ ]:
dev_ede_df2 =dev_ede_df.copy()

In [ ]:
dev_ede_df2[dev_ede_df2['serialnumber']=='575731312']

In [ ]:
dev_ede_df2 = pd.concat([dev_ede_df2, dev_ede_df2[dev_ede_df2['serialnumber']=='575731312']])

In [ ]:
dev_ede_df2[dev_ede_df2['serialnumber']=='575731312']

In [ ]:
start = time.time()
idk = dev_ede_df2.groupby(grp_by_full+['curated_reason']).size().to_frame()
idk = idk.reset_index()
hmm = idk.pivot(index=['serialnumber', 'aep_premise_nb', 'aep_event_dt'], columns='curated_reason').fillna(0)
hmm = hmm.droplevel(0, axis=1)
print(time.time()-start)

In [ ]:
dev_ede_df2.groupby(grp_by_full+['curated_reason']).size().to_frame()

In [ ]:
hmm

In [ ]:
rcpo

In [ ]:
hmm

In [ ]:
set(rcpo.columns.tolist()).symmetric_difference(set(hmm.columns.tolist()))

In [ ]:
rcpo[hmm.columns].equals(hmm)

In [ ]:
rcpo = rcpo[hmm.columns]

In [ ]:
diffs = Utilities_df.get_dfs_diff(rcpo.reset_index(), hmm.reset_index())

In [ ]:
diffs

In [ ]:

idk = dev_ede_df2.groupby(grp_by_full+['curated_reason']).size().to_frame()
idk = idk.reset_index()



In [ ]:
dev_ede_df2.groupby(grp_by_full, as_index=False, group_keys=False).agg(
    {
        'aep_premise_nb':lambda x: list(set(x)), 
        'serialnumber':lambda x: list(set(x))
    }
)

In [ ]:
start = time.time()
idk = dev_ede_df2.groupby(grp_by_full+['curated_reason']).size().to_frame()
idk = idk.reset_index()
hmm = idk.pivot(index=['serialnumber', 'aep_premise_nb', 'aep_event_dt'], columns='curated_reason').fillna(0)
hmm = hmm.droplevel(0, axis=1)
print(time.time()-start)

In [ ]:
hmm

In [ ]:
start = time.time()
idk = ede_df.groupby(grp_by_full+['curated_reason']).size().to_frame()
idk = idk.reset_index()
hmm = idk.pivot(index=['serialnumber', 'aep_premise_nb', 'aep_event_dt'], columns='curated_reason').fillna(0)
hmm = hmm.droplevel(0, axis=1)
print(time.time()-start)

In [ ]:
hmm

In [6]:
def get_reason_counts_per_group_QUICK(
        end_events_df, 
        group_cols=['serialnumber', 'aep_premise_nb'], 
        group_freq=None, 
        gpby_dropna=True, 
        reason_col='reason'
):
    r"""
    """
    #-------------------------
    # String and tuple (for MultiIndex columns) used for single group_col, whereas
    #   list used for multiple
    assert(Utilities.is_object_one_of_types(group_cols, [str, list, tuple]))
    if not isinstance(group_cols, list):
        group_cols = [group_cols]
    #-------------------------
    tmp_idx_col=None #Only set if group_freq is str, in which case grouping index
    if group_freq is not None:
        assert(Utilities.is_object_one_of_types(group_freq, [str, pd.core.resample.TimeGrouper]))
        if isinstance(group_freq, str):
            # Grouping by time on index!  
            # If one wants to group by time on column, need to supply full pd.Grouper object with 
            #   key argument equal to column
            # HOWEVER, functionality is easier here when grouping by column instead of index, so reset_index
            #   making it a column
            tmp_idx_col = Utilities.generate_random_string()
            assert(end_events_df.index.nlevels==1)
            end_events_df = end_events_df.reset_index(drop=False, names=tmp_idx_col)
            group_freq = pd.Grouper(freq=group_freq, key=tmp_idx_col)
        #-----
        grp_by_full = group_cols+[group_freq]
    else:
        grp_by_full = group_cols
    #-------------------------
    # First, get the number of counts in each group for each reason
    # Must add reason to grp_by_full, if not already (set operation to ensure not added twice)
    grp_by_full = list(set(grp_by_full+[reason_col]))
    
    # The result of this operation will be a DF with columns equal to grp_by_full+'size'
    return_df = end_events_df.groupby(grp_by_full, dropna=gpby_dropna, as_index=False).size()
    
    #-------------------------
    # Second, pivot about the reason column (with indices equal to all others in grp_by_full)
    # So, reason_col must be removed form grp_by_full
    # Also, if group_freq included, the item in grp_by_full needs to be changed from the pd.Grouper object
    #   to the column used for grouping (i.e., to group_freq.key)
    # NOTE: This will make the columns MultiIndex, with level_0 values equal to 'size' and level_1
    #       values equal to the reasons.
    # Any groups missing counts for various reasons will be filled with nan, hence the need for fillna(0)
    grp_by_full.remove(reason_col)
    if group_freq is not None:
        grp_by_full.remove(group_freq)
        grp_by_full.append(group_freq.key)
    return_df = return_df.pivot(index=grp_by_full, columns=reason_col).fillna(0)
    #-----
    # Drop level 0 from columns (all values = 'size', as stated above)
    # This will leave the reason columns.  However, this will give return_df.columns.name = reason_col, which
    #   will be confusing after we reset the index to include the grp_by_full values.
    # So, also rename the columns to None
    return_df = return_df.droplevel(0, axis=1)
    return_df.columns.name = None
    return_df = return_df.reset_index()
    
    #-------------------------
    return return_df

In [7]:
save_dir_base_rcpo = r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\regex_check\RCPO_Quick'

In [8]:
freq='D'
group_cols=['serialnumber', 'aep_premise_nb']
group_freq=pd.Grouper(freq=freq, key='aep_event_dt')

for opco in opcos:
    for date_range in date_ranges:
        save_name = f"end_events_{opco}_{date_range[0].replace('-','')}_{date_range[1].replace('-','')}.pkl"
        ede_df = pd.read_pickle(os.path.join(save_dir_base_cur, save_name))
        ede_df['aep_event_dt'] = pd.to_datetime(ede_df['aep_event_dt'])
        #-----
        rcpo_df = get_reason_counts_per_group_QUICK(
            end_events_df=ede_df, 
            group_cols=group_cols, 
            group_freq=group_freq, 
            gpby_dropna=True, 
            reason_col='curated_reason'
        )
        #-----
        rcpo_df.to_pickle(os.path.join(save_dir_base_rcpo, save_name))